In [ ]:
import os
import wandb
import halmos
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=1.5, style='whitegrid', palette=sns.color_palette('Set1'))

In [ ]:
keys = ['train/total_ts', 'train/mll', 'val/rmse', 'test/rmse']
config_keys = ['method', 'dataset']

In [ ]:
## KeOps
_, metrics1 = halmos.metrics('snym/bilateral-gp-experiments/qczouufb', keys=keys, config_keys=config_keys)

## Simplex-GP
_, metrics2 = halmos.metrics('snym/bilateral-gp-experiments/lcj8po92', keys=keys, config_keys=config_keys)

metrics = pd.concat([metrics1, metrics2])
metrics

In [ ]:
metrics['train/total_cu_ts'] = metrics.groupby(by=['run_id'])['train/total_ts'].cumsum()

In [ ]:
fig, axes = plt.subplots(figsize=(10, 10), nrows=2, ncols=2)

dataset = 'houseelectric'
plt_metrics = metrics[(metrics.dataset == dataset) & (metrics._step <= 15)]
sns.lineplot(data=plt_metrics, x='_step', y='train/mll', hue='method', ci=None, ax=axes[0,0])
sns.lineplot(data=plt_metrics, x='_step', y='train/total_cu_ts', hue='method', ci=None, ax=axes[0,1])
sns.lineplot(data=plt_metrics, x='_step', y='val/rmse', hue='method', ci=None, ax=axes[1,0])
sns.lineplot(data=plt_metrics, x='_step', y='test/rmse', hue='method', ci=None, ax=axes[1,1])

# axes[1,0].set_ylim([0.0, 2.0])
# axes[1,1].set_ylim([0.0, 2.0])

fig.tight_layout()

# ax.set_ylim([0., 2.])
fig.savefig(f'{dataset}_clipped.pdf')